In [0]:
%python
from pyspark.sql.functions import split, size,max

##df = spark.read.format('text').option('compression', 'gzip').load('s3a://consumer-orbit-us-east-1-holding/October2024_TCI/*.gz')
#df = spark.read.format('text').option('compression', 'gzip').load('s3://consumer-orbit-us-east-1-holding/October2024_TCI/*.gz')

#df = spark.read.format('text').option('compression', 'gzip').load('s3://tenetic-verisk-ingest-us-east-1/from-infutor/onprem/TCI_EMAIL/TCI_EMAIL_00.txt.gz')

df = spark.read.format('text').option('compression', 'gzip').load("/FileStore/tables/TCI_AK_txt-1.gz")
df_split = df.withColumn("split_col", split(df["value"], "\t"))

df_split_size=df_split.select('split_col',size('split_col').alias("No_of_Columns"))

MaxNumElements=df_split_size.agg({'No_of_Columns': 'max'}).collect()[0][0]





In [0]:
df_split_size.display()

In [0]:
def arraySplitIntoCols (df,No_of_Columns):
    for i in range(No_of_Columns):
        df = df.withColumn(f"col{i}", df.split_col[i])
    
    return df    

df_data=arraySplitIntoCols(df_split_size,MaxNumElements)

In [0]:
df_data=df_data.drop('split_col').drop('No_of_Columns')  

In [0]:
df_data.display()

In [0]:
Total_Consumer_Insights_Counts_df = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('/FileStore/tables/Total_Consumer_Insights_Counts__9____Copy__1_.xlsx')
    #.load('s3://consumer-orbit-us-east-1-holding/Total_Consumer_Insights_Counts (9) - Copy.xlsx')
    


In [0]:
df_Insight_Plus = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('/FileStore/tables/Total_Consumer_Insights_Plus_Counts__5___1_.xlsx')
    #.load('s3://consumer-orbit-us-east-1-holding/Total_Consumer_Insights_Plus_Counts (5) (1).xlsx')

display(df_Insight_Plus)

In [0]:
FieldNames_Total_Consumer_Insights_Counts_df=Total_Consumer_Insights_Counts_df.dropna(subset=['_c1'])
FieldNames_Total_Consumer_Insights_Counts_df = FieldNames_Total_Consumer_Insights_Counts_df.select('_c1')
FieldNames_Total_Consumer_Insights_Counts_df.display()

In [0]:
FieldNames_df_Insight_Plus=df_Insight_Plus.dropna(subset=['_c1'])
FieldNames_df_Insight_Plus = FieldNames_df_Insight_Plus.select('_c1')
FieldNames_df_Insight_Plus.display()

In [0]:
df_metadata=FieldNames_Total_Consumer_Insights_Counts_df.union(FieldNames_df_Insight_Plus)

In [0]:
# Take 'Field Name' column of second file (df2) and use it as header in first file (df1)

# Drop rows where 'Field Name' column is null
df_metadata=df_metadata.dropna(subset=['_c1'])

# Select only 'Field Name' column
df_metadata = df_metadata.select('_c1')


################################################################
#Removing the duplicates 


from pyspark.sql.functions import monotonically_increasing_id

# Add an index column to preserve the order
df_metadata = df_metadata.withColumn("index", monotonically_increasing_id())

# Drop duplicates based on the desired column(s) while keeping the first occurrence
df_metadata = df_metadata.dropDuplicates(['_c1'])
df_metadata=df_metadata.orderBy('index')


# Remove the index column
df_metadata = df_metadata.drop("index")


#####################################################################
display(df_metadata)




In [0]:
%python
# Extract the first row to use as header
new_header = df_metadata.first()

# Convert the first row to a list of column names
new_header = [str(col) for col in new_header]

# Apply the new header to df1
df_metadata = df_metadata.toDF(*new_header)


df_metadata = df_metadata.filter(df_metadata['Field Name'] != new_header[0])

df_metadata =df_metadata.dropDuplicates()

display(df_metadata)

In [0]:
#
# Use df_metadata  as header for df_data
new_header = [row['Field Name'] for row in df_metadata.collect()]
df_data_with_header = df_data.toDF(*new_header)

display(df_data_with_header)